# Reading events from RGTDB

Query to get JSON:

https://rgtdb.com/events/json?search=&offset=0&limit=100

This reads upcoming events from rgtdb.com and converts them into iCAL format and writes to a file for manual import. Then this saves or updates the events in a Google calendar.

## Get data and cache response

In [546]:
import pandas as pd 

import datetime
from datetime import datetime as dt

import io
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Debug level logging turned on")

import requests
from cachecontrol import CacheControl
from cachecontrol.caches import FileCache
from cachecontrol.heuristics import ExpiresAfter

sess = requests.session()
cached_sess = CacheControl(sess, cache = FileCache('.web_cache'), heuristic=ExpiresAfter(hours=1))

try:
    response = cached_sess.get('https://rgtdb.com/events/json?search=&offset=0&limit=200') # Get 200 events. Should be about a week's worth of events
    response.raise_for_status()

except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')
except Exception as err:
    print(f'Other error occurred: {err}')

logger.setLevel(logging.ERROR)

DEBUG:root:Debug level logging turned on
DEBUG:cachecontrol.controller:Looking up "https://rgtdb.com/events/json?search=&offset=0&limit=200" in the cache
DEBUG:cachecontrol.controller:No cache entry available
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rgtdb.com:443
DEBUG:urllib3.connectionpool:https://rgtdb.com:443 "GET /events/json?search=&offset=0&limit=200 HTTP/1.1" 200 None
DEBUG:cachecontrol.controller:Updating cache with response from "https://rgtdb.com/events/json?search=&offset=0&limit=200"
DEBUG:cachecontrol.controller:Caching b/c of expires header


In [547]:
response.json().keys()

dict_keys(['total', 'rows'])

## Convert JSON to Pandas Dataframe

Not necessary, but hey, pretending to be a data scientist feels cool.

In [548]:
df = pd.json_normalize(response.json(), 'rows')
df

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
0,Saturday Ladies Ride,02-20 12:30,/events/69308,[groupride],7,29.71 km,521 m,429 m,#100 South Coast,/courses/136342,False
1,Gimbels NY Long,02-20 13:00,/events/67309,[groupride],41,57.09 km,674 m,747 m,Gimbels NY Long,/courses/123049,False
2,Maratona Challenge Part 1,02-20 13:00,/events/57161,[groupride],51,55.04 km,1.76 km,1.67 km,Maratona Challenge 1,/courses/111085,False
3,Weekend Warrior,02-20 14:00,/events/59844,[groupride],3,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False
4,NCRA Old Race 7,02-20 14:00,/events/69405,[race],27,45.08 km,322 m,322 m,NCRA Old,/courses/130641,False
...,...,...,...,...,...,...,...,...,...,...,...
195,#100 Dales Part 3,03-05 19:30,/events/62272,[groupride],1,33.34 km,1.15 km,990 m,#100 Dales Part 3,/courses/111601,False
196,Taith de Cymru Stage1 ITT,03-06 00:30,/events/52696,"[race, itt]",3,10.64 km,145 m,142 m,TdC Stage 1 Prologue,/courses/95068,True
197,Taith de Cymru Stage1 ITT,03-06 06:30,/events/52697,"[race, itt]",3,10.64 km,145 m,142 m,TdC Stage 1 Prologue,/courses/95068,True
198,Virtuslo Medio Fondo,03-06 09:00,/events/49803,[race],6,99.06 km,983 m,983 m,MF Panonia,/courses/94144,False


## Convert start time into datetime format - and guess at the year

This will be an issue every year around new year.

In [549]:

this_year = str(dt.today().year)
df['date'] = pd.to_datetime(df['startAt'] + ' ' + this_year, format='%m-%d %H:%M %Y', utc=True)
df.set_index('date', inplace=True)


In [550]:
df.head()

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked
date,,,,,,,,,,,
2021-02-20 12:30:00+00:00,Saturday Ladies Ride,02-20 12:30,/events/69308,[groupride],7,29.71 km,521 m,429 m,#100 South Coast,/courses/136342,False
2021-02-20 13:00:00+00:00,Gimbels NY Long,02-20 13:00,/events/67309,[groupride],41,57.09 km,674 m,747 m,Gimbels NY Long,/courses/123049,False
2021-02-20 13:00:00+00:00,Maratona Challenge Part 1,02-20 13:00,/events/57161,[groupride],51,55.04 km,1.76 km,1.67 km,Maratona Challenge 1,/courses/111085,False
2021-02-20 14:00:00+00:00,Weekend Warrior,02-20 14:00,/events/59844,[groupride],3,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False
2021-02-20 14:00:00+00:00,NCRA Old Race 7,02-20 14:00,/events/69405,[race],27,45.08 km,322 m,322 m,NCRA Old,/courses/130641,False


## Use icalendar package to create ICAL format events

* GitHub: https://github.com/collective/icalendar


In [551]:
from datetime import timedelta
from icalendar import vCalAddress, vText
from icalendar import Calendar, Event
import pytz

cal = Calendar()
cal.add('prodid', '-//My calendar product//mxm.com//')
cal.add('version', '2.0')

for index, row in df.iterrows():
    print(index, row['name'], row['tags'])
    event = Event()
    event['uid'] = row['detailsUrl']
    event.add('summary', str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']))
    event.add('dtstart', index)
    event.add('dtend', index  + timedelta(hours=1))
    event.add('url', 'https://rgtdb.com' + row['detailsUrl'])
    event.add('description', row['distance'] + ' ' + 'https://rgtdb.com' + row['detailsUrl'])
    event.add('color', 'Tomato')
    event['location'] = vText(row['roadName'])

    cal.add_component(event)

2021-02-20 12:30:00+00:00 Saturday Ladies Ride ['groupride']
2021-02-20 13:00:00+00:00 Gimbels NY Long ['groupride']
2021-02-20 13:00:00+00:00 Maratona Challenge Part 1 ['groupride']
2021-02-20 14:00:00+00:00 Weekend Warrior ['groupride']
2021-02-20 14:00:00+00:00 NCRA Old Race 7 ['race']
2021-02-20 14:00:00+00:00 J2/9 TT ['race', 'itt']
2021-02-20 15:00:00+00:00 Lou's Saturday Group Ride ['groupride']
2021-02-20 15:00:00+00:00 Chain Gang ['groupride']
2021-02-20 15:00:00+00:00 Napoleon Dolomite ['race']
2021-02-20 15:30:00+00:00 OTR Flemish For Beginners ['race']
2021-02-20 16:00:00+00:00 Vueltina Asturias Stage 2 ['race']
2021-02-20 16:00:00+00:00 Maratona Transition Stage ['groupride']
2021-02-20 17:00:00+00:00 Maratona Challenge 2_1 ['groupride']
2021-02-20 17:10:00+00:00 Echelon Pro Women ['race', 'pro']
2021-02-20 17:15:00+00:00 Maratona Transition Stage ['groupride']
2021-02-20 18:00:00+00:00 Flat Out Flyer ['race']
2021-02-20 18:40:00+00:00 Echelon Pro Men ['race', 'pro']
2021-

## Write to File

Can use to manually import into Google, other calendars

In [552]:
import tempfile, os
f = open('./rgt_events.ics', 'wb')
f.write(cal.to_ical())
f.close()

## Use gcsa for Simplified Access to Google Calendar API

* GitHub: https://github.com/kuzmoyev/google-calendar-simple-api
* Docs: https://google-calendar-simple-api.readthedocs.io/en/latest/index.html

Need to add socket timeout of 5 minutes due to slow response on my machine

### Shared Calendar

* Calendar ID: 3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com
* Google sharing link: https://calendar.google.com/calendar/u/0?cid=M2U4Z2F1OGJvbW1mamszM2o5MnJ2NWs3cTBAZ3JvdXAuY2FsZW5kYXIuZ29vZ2xlLmNvbQ
* ICS format for e.g., Apple Calendar: https://calendar.google.com/calendar/ical/3e8gau8bommfjk33j92rv5k7q0%40group.calendar.google.com/public/basic.ics


In [553]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = '3e8gau8bommfjk33j92rv5k7q0@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


## Cleanup: Delete All Events from Google Calendar

Uncomment to clean up calendar.

In [554]:
#calendar.clear() # This gives an error in the Google API

#for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
#    print('Deleting:', event, event.event_id)
#    calendar.delete_event(event)

## Find existing events, mark for update instead of creation

In [555]:
import re

reExtractName = re.compile(" \[\'.*")

df['cal_id'] = None
df['event_obj'] = None
df['found'] = False

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):

    print(event)

    rideName = reExtractName.sub("", event.summary) # Get substring from event summary with just the name

    df.loc[(df['name'] == rideName) & (df.index == event.start), ['found', 'cal_id', 'event_obj']] = [True, event.event_id, event]


2021-02-19 15:00:00+00:00 - Napoleon Dolomite ['race'] 1
2021-02-19 15:00:00+00:00 - Chat Laps ['groupride'] 1
2021-02-19 15:00:00+00:00 - Race to the Light House ['race'] 3
2021-02-19 18:00:00+00:00 - Race to the Light House ['race'] 2
2021-02-19 18:00:00+00:00 - SDW Friday Frenzy #2 ['race'] 36
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 1 ['elimination', 'race'] 1
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 2 ['elimination', 'race'] 1
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 3 ['elimination', 'race'] 4
2021-02-19 19:30:00+00:00 - WKG’s Snap-dragon. Div 4 ['elimination', 'race'] 7
2021-02-19 22:00:00+00:00 - Race to the Light House ['race'] 1/22.43 km/515 m
2021-02-20 00:00:00+00:00 - Chain Gang ['groupride'] 2/25.16 km/614 m
2021-02-20 01:00:00+00:00 - London Calling ['race'] 2/25.88 km/133 m
2021-02-20 07:00:00+00:00 - Breakfast Club ['groupride'] 2/30.87 km/76 m
2021-02-20 08:00:00+00:00 - 11 cities ride 3/3 race ['race'] 4/93.51 km/91 m
2021-02-20 13:00:00

In [556]:
df.loc[df['found'] == True]


,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-20 12:30:00+00:00,Saturday Ladies Ride,02-20 12:30,/events/69308,[groupride],7,29.71 km,521 m,429 m,#100 South Coast,/courses/136342,False,ocfm26vobipt04mn5ku9v7dmt8,2021-02-20 12:30:00+00:00 - Saturday Ladies Ri...,True
2021-02-20 13:00:00+00:00,Gimbels NY Long,02-20 13:00,/events/67309,[groupride],41,57.09 km,674 m,747 m,Gimbels NY Long,/courses/123049,False,3pnnc4vkn80s5f82fet2mln398,2021-02-20 13:00:00+00:00 - Gimbels NY Long ['...,True
2021-02-20 13:00:00+00:00,Maratona Challenge Part 1,02-20 13:00,/events/57161,[groupride],51,55.04 km,1.76 km,1.67 km,Maratona Challenge 1,/courses/111085,False,kr5cscofbdi0p3i4t46g3bmqp8,2021-02-20 13:00:00+00:00 - Maratona Challenge...,True
2021-02-20 14:00:00+00:00,Weekend Warrior,02-20 14:00,/events/59844,[groupride],3,22.43 km,515 m,367 m,Cap Formentor,/courses/106,False,gl9sbfu3nlbu5adnjhs2n1vceg,2021-02-20 14:00:00+00:00 - Weekend Warrior ['...,True
2021-02-20 14:00:00+00:00,NCRA Old Race 7,02-20 14:00,/events/69405,[race],27,45.08 km,322 m,322 m,NCRA Old,/courses/130641,False,95tthmrarb3as600fog0fnslm0,2021-02-20 14:00:00+00:00 - NCRA Old Race 7 ['...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-23 07:00:00+00:00,Breakfast Club,02-23 07:00,/events/59689,[groupride],1,30.87 km,76 m,76 m,Borrego Springs,/courses/97,False,0c0g80jf9ecb5abh61pg1jmajs,2021-02-23 07:00:00+00:00 - Breakfast Club ['g...,True
2021-02-23 09:00:00+00:00,Gran Premio Pienza,02-23 09:00,/events/59255,[race],2,24.63 km,706 m,706 m,Pienza,/courses/64,False,6t7d6qd9pjrs97r7nd05958glk,2021-02-23 09:00:00+00:00 - Gran Premio Pienza...,True
2021-02-23 10:00:00+00:00,Chat Laps,02-23 10:00,/events/59750,[groupride],2,24.63 km,706 m,706 m,Pienza,/courses/64,False,ed2b34nbkpajf4n5o7ti42uh4c,2021-02-23 10:00:00+00:00 - Chat Laps ['groupr...,True


In [557]:
df.loc[df['found'] == False]

,name,startAt,detailsUrl,tags,signUps,distance,elevationGain,elevationLost,roadName,roadDetailsUrl,ranked,cal_id,event_obj,found
date,,,,,,,,,,,,,,
2021-02-23 15:00:00+00:00,Chat Laps,02-23 15:00,/events/59770,[groupride],1,24.63 km,706 m,706 m,Pienza,/courses/64,False,None,None,False
2021-02-23 15:00:00+00:00,Napoleon Dolomite,02-23 15:00,/events/59406,[race],1,14.08 km,1 m,1.11 km,Passo Dello Stelvio,/courses/46,False,None,None,False
2021-02-23 15:00:00+00:00,Gran Premio Pienza,02-23 15:00,/events/59259,[race],2,24.63 km,706 m,706 m,Pienza,/courses/64,False,None,None,False
2021-02-23 18:00:00+00:00,Gran Premio Pienza,02-23 18:00,/events/59263,[race],3,24.63 km,706 m,706 m,Pienza,/courses/64,False,None,None,False
2021-02-23 18:00:00+00:00,GFNS E-FONDO MARYLAND,02-23 18:00,/events/68035,[race],9,48.01 km,998 m,850 m,7. E Fondo Maryland V3,/courses/109201,False,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-05 19:30:00+00:00,#100 Dales Part 3,03-05 19:30,/events/62272,[groupride],1,33.34 km,1.15 km,990 m,#100 Dales Part 3,/courses/111601,False,None,None,False
2021-03-06 00:30:00+00:00,Taith de Cymru Stage1 ITT,03-06 00:30,/events/52696,"[race, itt]",3,10.64 km,145 m,142 m,TdC Stage 1 Prologue,/courses/95068,True,None,None,False
2021-03-06 06:30:00+00:00,Taith de Cymru Stage1 ITT,03-06 06:30,/events/52697,"[race, itt]",3,10.64 km,145 m,142 m,TdC Stage 1 Prologue,/courses/95068,True,None,None,False


## Add All Events to Google Calendar

In [558]:
from gcsa.event import Event as gcEvent

for index, row in df.iterrows():

#    if row['cal_id'] == None: # Easier to understand?
#        isNewEvent = True
#    else:
#        isNewEvent = False

#    isNewEvent = (row['cal_id'] == None) # Which is easier to understand?

    evntSummary = str(str(row['name']) + ' ' + str(row['tags']) + ' ' + str(row['signUps']) + '/' + row['distance'] + '/' + row['elevationGain'])

    evntDescription = 'Signups: ' +  str(row['signUps']) + '\n' + 'Distance: ' + row['distance'] + '\n' +  'Elevation gain: ' + row['elevationGain'] + '\n' + 'Descent: ' + row['elevationLost'] + '\n' + 'https://rgtdb.com' + str(row['detailsUrl'])

    if row['found'] == False:

        evntString = row['detailsUrl'].replace('/events/', '')

        print('+New event: ', index, row['name'], row['tags'], evntString)

        evntColor = '1'

        if "groupride" in row['tags']:
            evntColor = '2'
        elif 'pro' in row['tags']:
            evntColor = '3'
        elif 'elimination' in row['tags']:
            evntColor = '4'
        elif "itt" in row['tags']:
            evntColor = '5'
        elif "race" in row['tags']:
            evntColor = '6'

        event = gcEvent(
            evntSummary,
            start=index,
            timezone='UTC',
            location=str(row['roadName']),
            description=evntDescription,
            event_id=evntString,
            color = evntColor
        )

        print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else:

        print('-Updating event: ', index, row['name'], row['tags'], eventString)

        event = row['event_obj']

        event.summary = evntSummary
        event.description = evntDescription

        calendar.update_event(event)

ride'] 64068
-Updating event:  2021-02-21 20:00:00+00:00 GFNS GOLDEN E-FONDO ['race'] 64068
-Updating event:  2021-02-21 22:00:00+00:00 Flat Out Flyer ['race'] 64068
-Updating event:  2021-02-22 00:00:00+00:00 Tempo Tester ['groupride'] 64068
-Updating event:  2021-02-22 01:00:00+00:00 Flat Out Flyer ['race'] 64068
-Updating event:  2021-02-22 07:00:00+00:00 Breakfast Club ['groupride'] 64068
-Updating event:  2021-02-22 08:30:00+00:00 Killer-manjaro Decent ['race'] 64068
-Updating event:  2021-02-22 09:00:00+00:00 Flat Out Flyer ['race'] 64068
-Updating event:  2021-02-22 10:00:00+00:00 Chat Laps ['groupride'] 64068
-Updating event:  2021-02-22 12:00:00+00:00 Breakfast Club ['groupride'] 64068
-Updating event:  2021-02-22 15:00:00+00:00 Napoleon Dolomite ['race'] 64068
-Updating event:  2021-02-22 15:00:00+00:00 Chat Laps ['groupride'] 64068
-Updating event:  2021-02-22 15:00:00+00:00 Flat Out Flyer ['race'] 64068
-Updating event:  2021-02-22 17:45:00+00:00 Cyclocross Monday ['race'] 